In [11]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [13]:
archetypes = pd.read_csv('/Users/austincoffelt/Documents/Player_Archetypes.csv')
archetypes['Year'] = archetypes['year'].astype('str')

#get dummies
dummies = pd.get_dummies(archetypes[['Year', 'Archetype']], drop_first = False, dtype = 'int')
archetypes = pd.concat([archetypes, dummies], axis = 1)
archetypes = archetypes.drop(['Unnamed: 0', 'year'], axis = 1)
archetypes['Year'] = archetypes['Year'].astype('int')
archetypes.shape

(4508, 38)

In [15]:
# read in box score stats
boxScore = pd.read_csv('/Users/austincoffelt/Documents/WNBA_box_score_data - Sheet1.csv')
boxScore = boxScore[boxScore['Team'] != 'TOT']

# capitalize names
boxScore['Player'] = boxScore['Player'].str.upper()
boxScore['Player'] = boxScore['Player'].str.rstrip('*')

# recategorize the positions
boxScore.loc[boxScore['Pos'] == 'G-F', 'Pos'] = 'SG'
boxScore.loc[boxScore['Pos'] == 'F-G', 'Pos'] = 'SG'
boxScore.loc[boxScore['Pos'] == 'C-F', 'Pos'] = 'PowerF'
boxScore.loc[boxScore['Pos'] == 'F-C', 'Pos'] = 'PowerF'

boxScore = boxScore.drop('G', axis = 1)
boxScore = boxScore.fillna(value = 0)

#get dummies
dummies = pd.get_dummies(boxScore['Pos'], drop_first = False, dtype = 'int')
boxScore = pd.concat([boxScore, dummies], axis = 1)

cont_columns = ['Player', 'Team', 'Pos', 'Year', 'FGA', '3PA', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'C', 'F', 'G', 'PowerF', 'SG']
boxScore = boxScore[cont_columns]

# fix merge problems
boxScore.loc[(boxScore['Team'] == 'SAS'), 'Team'] = 'SAN'
boxScore.loc[(boxScore['Player'] == 'TIFFANY JACKSON-JONES'), 'Player'] = 'TIFFANY JACKSON'
boxScore.loc[(boxScore['Player'] == 'TAMIKA WILLIAMS'), 'Player'] = 'TAMIKA RAYMOND'
boxScore.loc[(boxScore['Player'] == 'AZURA STEVENS'), 'Player'] = 'AZURÁ STEVENS'
boxScore.loc[(boxScore['Player'] == 'JAYNE APPEL-MARINELLI'), 'Player'] = 'JAYNE APPEL MARINELLI'
boxScore.loc[(boxScore['Player'] == 'VAL WHITING-RAYMOND'), 'Player'] = 'VAL WHITING'
boxScore.loc[(boxScore['Player'] == 'BRIANA GILBREATH-BUTLER'), 'Player'] = 'BRIANA BUTLER'
boxScore.loc[(boxScore['Player'] == 'ANNIE LA FLEUR'), 'Player'] = 'ANNIE BURGESS'
boxScore.loc[(boxScore['Player'] == 'AMANDA ZAHUI B.'), 'Player'] = 'AMANDA ZAHUI B'
boxScore.loc[(boxScore['Player'] == 'PAMELA MCGEE'), 'Player'] = 'PAM MCGEE'
boxScore.loc[(boxScore['Player'] == 'MARINE JOHANNES'), 'Player'] = 'MARINE JOHANNÈS'
boxScore.loc[(boxScore['Player'] == 'BRITTANY BOYD'), 'Player'] = 'BRITTANY BOYD-JONES'
boxScore.loc[(boxScore['Player'] == 'ADRIANA MOISES'), 'Player'] = 'ADRIANA MOISES PINTO'
boxScore.loc[(boxScore['Player'] == 'ANEIKA HENRY-MORELLO'), 'Player'] = 'ANEIKA MORELLO'
boxScore.loc[(boxScore['Player'] == 'NIKOLINA MILIC'), 'Player'] = 'NIKOLINA MILIĆ'
boxScore.loc[(boxScore['Player'] == 'ALESSANDRA SANTOS DE OLIVEIRA'), 'Player'] = 'ALESSANDRA SANTOS DE OLIVE'
boxScore.loc[(boxScore['Player'] == 'AD DURR'), 'Player'] = 'ASIA (AD) DURR'
boxScore.loc[(boxScore['Player'] == 'SANDRA VAN EMBRICQS'), 'Player'] = 'SANDRA VANEMBRICQS'
boxScore.loc[(boxScore['Player'] == 'TEMI FAGBENLE'), 'Player'] = 'TEMI FÁGBÉNLÉ'
boxScore.loc[(boxScore['Player'] == 'MARIE GULICH'), 'Player'] = 'MARIE GÜLICH'
boxScore.loc[(boxScore['Player'] == 'TRENA TRICE'), 'Player'] = 'TRENA TRICE-HILL'
boxScore.loc[(boxScore['Player'] == 'LAQUANDA BARKSDALE'), 'Player'] = 'LAQUANDA QUICK'
boxScore.loc[(boxScore['Player'] == 'JOY HOLMES-HARRIS'), 'Player'] = 'JOY HOLMES'
boxScore.loc[(boxScore['Player'] == 'TAWONA ALHALEEM'), 'Player'] = 'TOWANA ALHALEEM'
boxScore.loc[(boxScore['Player'] == 'EVGENIYA BELYAKOVA'), 'Player'] = 'EVGENIIA BELYAKOVA'
boxScore.loc[(boxScore['Player'] == 'VEDRANA GRGIN-FONSECA'), 'Player'] = 'VEDRANA GRGIN FONSECA'
boxScore.loc[(boxScore['Player'] == 'SHARNEE ZOLL'), 'Player'] = 'SHARNEE ZOLL-NORMAN'
boxScore.loc[(boxScore['Player'] == 'RAEGAN PEBLEY'), 'Player'] = 'RAEGAN SCOTT'
boxScore.loc[(boxScore['Player'] == 'MIAO LIJIE'), 'Player'] = 'MIAO LI JIE'
boxScore.loc[(boxScore['Player'] == 'JUNG SUN-MIN'), 'Player'] = 'SUN MIN JUNG'
boxScore.loc[(boxScore['Player'] == 'BERNADETT HATAR'), 'Player'] = 'BERNADETT HATÁR'

boxScore.shape

(4727, 20)

In [17]:
boxScore[boxScore.duplicated(['Player', 'Year'])]

,Player,Team,Pos,Year,FGA,3PA,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,C,F,G,PowerF,SG
169,QUEEN EGBO,LAS,PowerF,2024,12.6,0.0,0.0,12.6,37.9,0.0,12.6,0.0,0.0,12.6,25.2,0,0,0,1,0
170,QUEEN EGBO,LVA,PowerF,2024,16.7,0.0,0.0,2.8,13.9,0.0,5.6,2.8,0.0,5.6,22.3,0,0,0,1,0
171,MORGAN BERTSCH,DAL,F,2024,6.8,2.3,4.5,2.3,11.4,0.0,2.3,0.0,9.1,9.1,2.3,0,1,0,0,0
172,MORIAH JEFFERSON,CHI,G,2024,10.7,4.1,2.5,1.2,2.9,9.9,0.8,0.0,2.5,3.7,5.0,0,0,1,0,0
173,ODYSSEY SIMS,LAS,G,2024,16.4,3.3,4.7,0.8,3.2,9.8,1.7,0.1,4.6,2.7,18.8,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702,OLYMPIA SCOTT,DET,C,1999,18.9,0.0,6.3,3.1,12.6,3.1,0.0,3.1,5.2,14.7,16.8,1,0,0,0,0
4833,ERIN ALEXANDER,UTA,G,1998,17.2,14.9,1.6,0.0,2.3,2.3,0.8,0.0,3.9,5.5,13.3,0,0,1,0,0
4834,TAMMY JACKSON,HOU,PowerF,1998,7.4,0.0,3.9,2.1,7.4,2.1,1.4,1.8,4.9,10.9,8.4,0,0,0,1,0
4835,PAULINE JORDAN,SAC,PowerF,1998,13.7,0.2,6.3,3.2,9.9,3.6,3.2,2.7,5.4,9.9,13.3,0,0,0,1,0


In [23]:
for year in range(1997, 2025):
    temp = pd.read_csv(f'/Users/austincoffelt/Documents/WNBA_shooting_location/WNBA_players_{year}_shooting_location.csv')
    temp['Year'] = year
    temp = temp.iloc[:, -22:]
    temp.to_csv(f'/Users/austincoffelt/Documents/WNBA_shooting_location/WNBA_players_{year}_shooting_location.csv')

In [25]:
# Generate the list of file paths
file_paths = [f'WNBA_shooting_location/WNBA_players_{year}_shooting_location.csv' for year in range(1997, 2025)]

# Concatenate all files into one DataFrame
shooting_location = pd.concat((pd.read_csv(f) for f in file_paths), ignore_index=True)
shooting_location = shooting_location.fillna(value = 0)

# grab only needed columns
cont_cols = ['RA_FG%', 'ITP_FG%', 'MR_FG%', 'LC3_FG%', 'RC3_FG%', 'AB3_FG%']
imp_cols = ['Player', 'Team', 'Year'] + cont_cols 
shooting_location = shooting_location.loc[:, imp_cols]

# fill na values
for col in cont_cols:
    shooting_location.loc[shooting_location[col] == '-', col] = 0

# change dtype to float
shooting_location[cont_cols] = shooting_location[cont_cols].astype('float')
shooting_location.shape

(4509, 9)

In [27]:
for year in range(1997, 2025):
    temp = pd.read_csv(f'/Users/austincoffelt/Documents/WNBA_players/WNBA_players_{year}.csv')
    temp['Year'] = year
    temp.to_csv(f'/Users/austincoffelt/Documents/WNBA_players/WNBA_players_{year}.csv')

In [29]:
# Generate the list of file paths
file_paths = [f'WNBA_players/WNBA_players_{year}.csv' for year in range(1997, 2025)]

# Concatenate all files into one DataFrame
true_shooting = pd.concat((pd.read_csv(f) for f in file_paths), ignore_index=True)
true_shooting = true_shooting.fillna(value = 0)

true_shooting['Player'] = true_shooting['Name']

true_shooting = true_shooting[['Player', 'Team', 'Year', 'Total Minutes', 'TS%']]

true_shooting = true_shooting.sort_values('Total Minutes', ascending = False)
true_shooting = true_shooting.drop_duplicates(subset = ['Player', 'Year'], keep = 'first')
true_shooting.shape

(4509, 5)

In [31]:
df = true_shooting.merge(boxScore, on = ['Player', 'Year'], how = 'inner')
df['Team'] = df['Team_x']
df = df.drop(['Team_x', 'Team_y'], axis = 1)

df = df.merge(shooting_location, on = ['Player', 'Year'], how = 'inner')
df['Team'] = df['Team_x']
df = df.drop(['Team_x', 'Team_y'], axis = 1)

df = df.merge(archetypes, on = ['Player', 'Year'], how = 'inner')
df['Team'] = df['Team_x']
df = df.drop(['Team_x', 'Team_y'], axis = 1)

df = df.sort_values('Total Minutes', ascending = False)
df = df.drop_duplicates(subset = ['Player', 'Year'], keep = 'first')
df.shape

(4495, 63)

In [33]:
ORAPM = pd.read_csv('/Users/austincoffelt/Documents/Bayesian_ORAPM.csv')
ORAPM.shape

(4367, 8)

In [35]:
DRAPM = pd.read_csv('/Users/austincoffelt/Documents/Bayesian_DRAPM.csv')
DRAPM.shape

(4482, 8)

In [37]:
dfORAPM = df.merge(ORAPM, on = ['Player', 'Year'], how = 'inner')
dfORAPM['Team'] = dfORAPM['Team_x']
dfORAPM = dfORAPM.drop(['Team_x', 'Team_y'], axis = 1)

dfORAPM = dfORAPM.sort_values('Total Minutes', ascending = False)
dfORAPM = dfORAPM.drop_duplicates(subset = ['Player', 'Year'], keep = 'first')
dfORAPM.shape

(4354, 68)

In [39]:
dfDRAPM = df.merge(DRAPM, on = ['Player', 'Year'], how = 'inner')
dfDRAPM['Team'] = dfDRAPM['Team_x']
dfDRAPM = dfDRAPM.drop(['Team_x', 'Team_y'], axis = 1)

dfDRAPM = dfDRAPM.sort_values('Total Minutes', ascending = False)
dfDRAPM = dfDRAPM.drop_duplicates(subset = ['Player', 'Year'], keep = 'first')
dfDRAPM.shape

(4468, 68)

In [41]:
X = dfORAPM.drop(['lower', 'upper', 'minutes', 'G', 'Year_2024', 'Archetype_Spot-up Shooter'], axis = 1)
X['intercept'] = 1
X.shape

(4354, 63)

In [43]:
Y = dfORAPM.loc[dfORAPM['reliability'] == 'high', 'mean']
Y = pd.DataFrame(Y)

In [45]:
def get_interactions(dataframe, cont_cols, dis_cols):

    for col1 in dis_cols:
        for col2 in cont_cols:
            new_col = f'{col1}({col2})'
            dataframe[new_col] = dataframe[col1] * dataframe[col2]

In [47]:
get_interactions(X,
                ['FGA', '3PA', 'FTA',
                'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%'], 
                ['C', 'PowerF', 'F', 'SG', 'Archetype_Interior Primary Option', 'Archetype_Perimeter-Oriented Big/Wing',
                 'Archetype_Primary On-Ball Creator', 'Archetype_Roller/Cutter', 'Archetype_Secondary Creator/Distributor'])

X.shape

(4354, 171)

In [49]:
X_high_reliability = X[X['reliability'] == 'high']
X = X.drop('reliability', axis = 1)
X_high_reliability = X_high_reliability.drop('reliability', axis = 1)
X_high_reliability.shape

(2211, 170)

In [51]:
mod = sm.OLS(Y, X_high_reliability.drop(['Player', 'Team', 'Year', 'Pos', 'Archetype', 'mean', 'Total Minutes'], axis = 1))
res = mod.fit()
robust_results = res.get_robustcov_results(cov_type='HC1')
robust_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mean   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     11.44
Date:                Tue, 12 Aug 2025   Prob (F-statistic):          8.78e-190
Time:                        11:33:02   Log-Likelihood:                -5498.2
No. Observations:                2211   AIC:                         1.132e+04
Df Residuals:                    2048   BIC:                         1.225e+04
Df Model:                         162                                         
Covariance Type:                  HC1                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
TS%                                              0.1737      0.115      1.516      0.130      -0.051       0.398
FGA                                              0.0787      0.319      0.246      0.805      -0.547       0.705
3PA                                             -0.1107      0.126     -0.878      0.380      -0.358       0.137
FTA                                             -0.2105      0.192     -1.098      0.272      -0.586       0.165
ORB                                              0.9076      0.465      1.951      0.051      -0.005       1.820
TRB                                             -0.1151      0.183     -0.628      0.530      -0.474       0.244
AST                                              0.7545      0.119      6.327      0.000       0.521       0.988
STL                                              0.4039      0.281      1.436      0.151      -0.148       0.955
BLK                                             -0.3369      0.538     -0.626      0.532      -1.393       0.719
TOV                                             -1.5378      0.210     -7.318      0.000      -1.950      -1.126
PF                                              -0.3563      0.173     -2.054      0.040      -0.696      -0.016
PTS                                              0.1837      0.292      0.630      0.529      -0.388       0.756
C                                                5.4767     12.027      0.455      0.649     -18.109      29.063
F                                               -4.1153      5.902     -0.697      0.486     -15.690       7.459
PowerF                                          14.6730     14.112      1.040      0.299     -13.002      42.348
SG                                              14.4722     12.357      1.171      0.242      -9.762      38.707
RA_FG%                                           0.0159      0.011      1.489      0.137      -0.005       0.037
ITP_FG%                                          0.0154      0.008      1.946      0.052      -0.000       0.031
MR_FG%                                          -0.0117      0.009     -1.347      0.178      -0.029       0.005
LC3_FG%                                          0.0024      0.003      0.794      0.427      -0.003       0.008
RC3_FG%                                         -0.0049      0.003     -1.902      0.057      -0.010       0.000
AB3_FG%                                         -0.0028      0.006     -0.461      0.645      -0.015       0.009
Year_1997                                        3.4349      0.588      5.841      0.000       2.282       4.588
Year_1998                                        2.2126      0.603      3.670      0.000       1.030       3.395
Year_1999                       

In [59]:
# Get coefficients and p-values as pandas Series
coefficients = pd.Series(robust_results.params, name='Coefficient')
p_values = pd.Series(robust_results.pvalues, name='P-value')

# Combine into a DataFrame
results_df = pd.DataFrame({
    'Variable': coefficients.index,
    'Coefficient': coefficients.values,
    'P-value': p_values.values
})
results_df = results_df.sort_values(by = 'Variable', ascending = True)

names = list(X.drop(['Player', 'Team', 'Year', 'Pos', 'Archetype', 'mean', 'Total Minutes'], axis = 1).columns)
for i in range(len(coefficients.index)):
    results_df.loc[i, 'Variable_Names'] = names[i]

results_df = results_df.drop(list(range(16, 49)))
results_df.head()

,Variable,Coefficient,P-value,Variable_Names
0,0,0.173679,0.129696,TS%
1,1,0.078669,0.805372,FGA
2,2,-0.110741,0.379928,3PA
3,3,-0.210485,0.272368,FTA
4,4,0.907630,0.051154,ORB


In [61]:
OBPM_coefficients = pd.DataFrame(data=[results_df['Coefficient'].values], columns=results_df['Variable_Names'].values)
OBPM_coefficients

,TS%,FGA,3PA,FTA,ORB,TRB,AST,STL,BLK,TOV,...,Archetype_Secondary Creator/Distributor(FTA),Archetype_Secondary Creator/Distributor(ORB),Archetype_Secondary Creator/Distributor(TRB),Archetype_Secondary Creator/Distributor(AST),Archetype_Secondary Creator/Distributor(STL),Archetype_Secondary Creator/Distributor(BLK),Archetype_Secondary Creator/Distributor(TOV),Archetype_Secondary Creator/Distributor(PF),Archetype_Secondary Creator/Distributor(PTS),Archetype_Secondary Creator/Distributor(TS%)
0,0.173679,0.078669,-0.110741,-0.210485,0.90763,-0.115095,0.754517,0.403879,-0.33693,-1.53778,...,0.134588,-0.127898,0.000186,-0.127375,-0.231319,0.702613,0.603255,0.082318,-0.197456,0.122159


In [63]:
# Ensure X has the same columns as OBPM (and in the same order)
common_cols = [col for col in OBPM_coefficients.columns if col in X.columns]

# Calculate the linear combination: X * OBPM (dot product)
X['OBPM'] = X[common_cols] @ OBPM_coefficients[common_cols].T.squeeze()

In [65]:
dfOBPM = X[['Player', 'Year', 'Pos', 'Archetype', 'OBPM', 'mean']]
dfOBPM = dfOBPM.sort_values('OBPM', ascending=False)
dfOBPM.head(15)

,Player,Year,Pos,Archetype,OBPM,mean
4315,CARLA MCGHEE,2002,F,Interior Primary Option,15.427329,19.826325
4295,ALEXIS PRINCE,2018,F,Perimeter-Oriented Big/Wing,15.389260,38.928249
4326,DANA WYNNE,2001,F,Roller/Cutter,13.984800,5.421226
3934,ELENA DELLE DONNE,2021,SG,Perimeter-Oriented Big/Wing,10.535035,9.933903
4321,AMERYST ALSTON,2017,G,Interior Primary Option,9.465982,11.733970
4077,BRITTANY BOYD-JONES,2017,G,Interior Primary Option,9.250096,-8.564988
4212,JESSICA ADAIR,2010,C,Roller/Cutter,8.980722,55.090251
4187,ALEXIS PRINCE,2020,F,Spot-up Shooter,8.929539,-31.129254
4330,ADIA BARNES,2001,F,Roller/Cutter,8.656242,-19.217132
3987,RENNIA DAVIS,2022,SG,Roller/Cutter,8.396244,23.191778


In [67]:
careerOBPM = dfOBPM.groupby('Player')['OBPM'].mean()
careerOBPM = careerOBPM.sort_values(ascending = False)
careerOBPM.head(15)

Player
DANA WYNNE           13.984800
RENNIA DAVIS          8.396244
PAULINE JORDAN        6.802474
ALEXIS PRINCE         6.502167
TINA KRAJISNIK        6.036403
ELENA DELLE DONNE     5.635283
CORNELIA GAYDEN       4.865688
ANNELI MALEY          3.475443
BREANNA STEWART       3.443822
SPARKLE TAYLOR        2.957662
JONQUEL JONES         2.834664
MAYA MOORE            2.598960
TAYLOR MIKESELL       2.497536
JAYLYN SHERROD        2.013679
CYNTHIA COOPER        1.849146
Name: OBPM, dtype: float64

In [69]:
byPosition = dfOBPM.groupby('Pos')['OBPM'].mean()
byPosition

Pos
C        -5.559996
F        -4.543388
G        -4.476629
PowerF   -3.589956
SG       -3.087049
Name: OBPM, dtype: float64

In [71]:
byArchetype = dfOBPM.groupby('Archetype')['OBPM'].mean()
byArchetype

Archetype
Interior Primary Option         -3.332561
Perimeter-Oriented Big/Wing     -0.984219
Primary On-Ball Creator         -1.997317
Roller/Cutter                   -6.444382
Secondary Creator/Distributor   -5.639637
Spot-up Shooter                 -4.660706
Name: OBPM, dtype: float64

In [73]:
dfOBPM[dfOBPM['Player'] == 'DIANA TAURASI']

,Player,Year,Pos,Archetype,OBPM,mean
512,DIANA TAURASI,2018,G,Primary On-Ball Creator,4.432464,5.531506
2089,DIANA TAURASI,2020,G,Primary On-Ball Creator,3.692669,0.589387
400,DIANA TAURASI,2014,G,Primary On-Ball Creator,2.711970,3.331535
213,DIANA TAURASI,2008,G,Primary On-Ball Creator,2.662201,5.921617
369,DIANA TAURASI,2013,G,Primary On-Ball Creator,2.536341,5.383039
591,DIANA TAURASI,2009,G,Primary On-Ball Creator,2.493762,1.938861
140,DIANA TAURASI,2006,G,Primary On-Ball Creator,2.468922,10.418832
3448,DIANA TAURASI,2012,G,Primary On-Ball Creator,1.454118,6.976854
607,DIANA TAURASI,2011,G,Primary On-Ball Creator,1.431730,6.852974
394,DIANA TAURASI,2007,G,Primary On-Ball Creator,1.254732,5.964024


In [105]:
x = dfDRAPM.drop(['lower', 'upper', 'minutes', 'G', 'Year_2024', 'Archetype_Spot-up Shooter'], axis = 1)
x['intercept'] = 1
x.shape

(4468, 63)

In [107]:
y = dfDRAPM.loc[dfDRAPM['reliability'] == 'high', 'mean']
y = pd.DataFrame(y)
y.shape

(2266, 1)

In [109]:
get_interactions(x,
                ['FGA', '3PA', 'FTA',
                'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%'], 
                ['C', 'PowerF', 'F', 'SG', 'Archetype_Interior Primary Option', 'Archetype_Perimeter-Oriented Big/Wing',
                 'Archetype_Primary On-Ball Creator', 'Archetype_Roller/Cutter', 'Archetype_Secondary Creator/Distributor'])

x.shape

(4468, 171)

In [111]:
x_high_reliability = x[x['reliability'] == 'high']
x = x.drop('reliability', axis = 1)
x_high_reliability = x_high_reliability.drop('reliability', axis = 1)
x_high_reliability.shape

(2266, 170)

In [113]:
mod1 = sm.OLS(y, x_high_reliability.drop(['Player', 'Team', 'Year', 'Pos', 'Archetype', 'mean', 'Total Minutes'], axis = 1))
res1 = mod1.fit()
robust_results1 = res1.get_robustcov_results(cov_type='HC1')
robust_results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mean   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     2.958
Date:                Tue, 12 Aug 2025   Prob (F-statistic):           1.28e-28
Time:                        11:49:54   Log-Likelihood:                -5673.8
No. Observations:                2266   AIC:                         1.167e+04
Df Residuals:                    2103   BIC:                         1.261e+04
Df Model:                         162                                         
Covariance Type:                  HC1                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
TS%                                             -0.0819      0.106     -0.772      0.440      -0.290       0.126
FGA                                             -0.4209      0.314     -1.339      0.181      -1.037       0.195
3PA                                              0.1938      0.123      1.573      0.116      -0.048       0.435
FTA                                              0.3406      0.194      1.751      0.080      -0.041       0.722
ORB                                              0.1023      0.426      0.240      0.810      -0.733       0.938
TRB                                              0.1546      0.164      0.940      0.347      -0.168       0.477
AST                                             -0.0971      0.117     -0.829      0.407      -0.327       0.133
STL                                              0.2395      0.263      0.912      0.362      -0.275       0.754
BLK                                              0.0962      0.544      0.177      0.860      -0.970       1.163
TOV                                             -0.0590      0.230     -0.257      0.797      -0.510       0.392
PF                                              -0.0451      0.170     -0.266      0.790      -0.378       0.288
PTS                                              0.1800      0.296      0.608      0.543      -0.401       0.760
C                                              -11.8708      9.081     -1.307      0.191     -29.679       5.937
F                                               -6.1043      5.968     -1.023      0.306     -17.807       5.599
PowerF                                         -13.2024     10.449     -1.264      0.207     -33.694       7.289
SG                                              -6.5229     12.107     -0.539      0.590     -30.267      17.221
RA_FG%                                           0.0112      0.010      1.139      0.255      -0.008       0.031
ITP_FG%                                          0.0130      0.008      1.698      0.090      -0.002       0.028
MR_FG%                                           0.0093      0.009      1.028      0.304      -0.008       0.027
LC3_FG%                                          0.0066      0.003      2.144      0.032       0.001       0.013
RC3_FG%                                         -0.0026      0.003     -0.953      0.341      -0.008       0.003
AB3_FG%                                         -0.0021      0.006     -0.380      0.704      -0.013       0.009
Year_1997                                       -0.2954      0.642     -0.460      0.646      -1.555       0.965
Year_1998                                        0.6540      0.683      0.957      0.339      -0.686       1.994
Year_1999                       

In [115]:
# Get coefficients and p-values as pandas Series
coefficients1 = pd.Series(robust_results1.params, name='Coefficient')
p_values1 = pd.Series(robust_results1.pvalues, name='P-value')

# Combine into a DataFrame
results_df1 = pd.DataFrame({
    'Variable': coefficients1.index,
    'Coefficient': coefficients1.values,
    'P-value': p_values1.values
})
results_df1 = results_df1.sort_values(by = 'Variable', ascending = True)

names1 = list(x.drop(['Player', 'Team', 'Year', 'Pos', 'Archetype', 'mean', 'Total Minutes'], axis = 1).columns)
for i in range(len(coefficients1.index)):
    results_df1.loc[i, 'Variable_Names'] = names1[i]

results_df1 = results_df1.drop(list(range(16, 49)))
results_df1.head()

,Variable,Coefficient,P-value,Variable_Names
0,0,-0.081892,0.440285,TS%
1,1,-0.420877,0.180684,FGA
2,2,0.193790,0.115927,3PA
3,3,0.340566,0.080022,FTA
4,4,0.102308,0.810233,ORB


In [117]:
DBPM_coefficients = pd.DataFrame(data=[results_df1['Coefficient'].values], columns=results_df1['Variable_Names'].values)
DBPM_coefficients

,TS%,FGA,3PA,FTA,ORB,TRB,AST,STL,BLK,TOV,...,Archetype_Secondary Creator/Distributor(FTA),Archetype_Secondary Creator/Distributor(ORB),Archetype_Secondary Creator/Distributor(TRB),Archetype_Secondary Creator/Distributor(AST),Archetype_Secondary Creator/Distributor(STL),Archetype_Secondary Creator/Distributor(BLK),Archetype_Secondary Creator/Distributor(TOV),Archetype_Secondary Creator/Distributor(PF),Archetype_Secondary Creator/Distributor(PTS),Archetype_Secondary Creator/Distributor(TS%)
0,-0.081892,-0.420877,0.19379,0.340566,0.102308,0.154562,-0.097082,0.239533,0.09617,-0.059045,...,-0.312285,-0.739961,0.038001,0.05632,0.40163,0.858151,0.153733,0.31447,0.155873,-0.081248


In [119]:
# Ensure X has the same columns as OBPM (and in the same order)
common_cols1 = [col1 for col1 in DBPM_coefficients.columns if col1 in x.columns]

# Calculate the linear combination: X * OBPM (dot product)
x['DBPM'] = x[common_cols1] @ DBPM_coefficients[common_cols1].T.squeeze()

In [121]:
dfDBPM = x[['Player', 'Year', 'Pos', 'Archetype', 'DBPM', 'mean']]
dfDBPM = dfDBPM.sort_values('DBPM', ascending=False)
dfDBPM.head(15)

,Player,Year,Pos,Archetype,DBPM,mean
4418,LINNAE HARPER,2021,G,Roller/Cutter,30.987868,19.357403
4467,ANGEL MCCOUGHTRY,2021,SG,Perimeter-Oriented Big/Wing,23.543793,102.584162
4147,JAMILA WIDEMAN,2000,G,Roller/Cutter,20.631103,-14.089955
4414,RENEE ROBINSON,2000,G,Spot-up Shooter,19.945049,-20.820116
4451,AMBER HARRIS,2018,F,Roller/Cutter,18.982657,-15.749554
4305,SOPHIE BRUNNER,2017,F,Roller/Cutter,18.946640,-21.824583
4431,MONIQUE AMBERS,2002,F,Roller/Cutter,18.842591,-3.550652
3453,ERICA MCCALL,2020,F,Roller/Cutter,17.386219,3.088372
4420,PLENETTE PIERSON,2009,F,Roller/Cutter,17.034200,-29.345445
4411,SHANTIA OWENS,2002,F,Roller/Cutter,16.931191,-28.726531


In [123]:
careerDBPM = dfDBPM.groupby('Player')['DBPM'].mean()
careerDBPM = careerDBPM.sort_values(ascending = False)
careerDBPM.head(15)

Player
RENEE ROBINSON        19.945049
SOPHIE BRUNNER        18.946640
BEVERLY WILLIAMS      15.610883
DANA WYNNE            15.235843
CORI CHAMBERS         15.074189
RENNIA DAVIS          14.888391
KIM GESSIG            14.686041
BRITANY MILLER        12.620412
MARCEDES WALKER       11.835048
JAKIA BROWN-TURNER    11.551155
MONIQUE COKER         11.318937
AMISHA CARTER         10.950389
HANNAH SJERVEN        10.320750
RYNELDI BECENTI       10.251535
ALEAH GOODMAN          9.436090
Name: DBPM, dtype: float64

In [125]:
byPosition1 = dfDBPM.groupby('Pos')['DBPM'].mean()
byPosition1

Pos
C        -0.020083
F        -0.338019
G        -0.477499
PowerF    0.303142
SG       -0.490135
Name: DBPM, dtype: float64

In [127]:
byArchetype1 = dfDBPM.groupby('Archetype')['DBPM'].mean()
byArchetype1

Archetype
Interior Primary Option         -0.307089
Perimeter-Oriented Big/Wing     -0.882201
Primary On-Ball Creator         -1.430067
Roller/Cutter                    0.537566
Secondary Creator/Distributor   -0.043005
Spot-up Shooter                 -0.517588
Name: DBPM, dtype: float64

In [129]:
dfDBPM[dfDBPM['Player'] == "CAITLIN CLARK"]

,Player,Year,Pos,Archetype,DBPM,mean
3,CAITLIN CLARK,2024,G,Primary On-Ball Creator,-2.172322,-0.918636


In [131]:
dfBPM = dfDBPM
dfBPM['BPM'] = dfBPM['DBPM'] + dfOBPM['OBPM']
dfBPM['RAPM'] = dfBPM['mean'] + dfOBPM['mean']
dfBPM = dfBPM.drop(['DBPM', 'mean'], axis = 1)
dfBPM = dfBPM.dropna()
dfBPM = dfBPM.sort_values('BPM', ascending=False)
dfBPM.head(15)

,Player,Year,Pos,Archetype,BPM,RAPM
4326,BLAKE DIETRICK,2016,G,Roller/Cutter,25.227910,12.176102
4331,AMISHA CARTER,2005,F,Roller/Cutter,18.956086,40.415577
4295,KYSRE GONDREZICK,2024,G,Spot-up Shooter,16.383247,69.209373
4239,SHAUNZINSKI GORTMAN,2004,G,Secondary Creator/Distributor,15.486769,16.136849
3453,ERICA MCCALL,2020,F,Roller/Cutter,11.577287,6.119560
4147,JAMILA WIDEMAN,2000,G,Roller/Cutter,11.436589,-35.103132
4316,MONIQUE COKER,2004,PowerF,Spot-up Shooter,10.625138,17.169778
4183,SHYLA HEAL,2021,G,Secondary Creator/Distributor,9.251625,27.820914
3917,KALANA GREENE,2014,G,Roller/Cutter,9.169432,1.949145
4187,ADRIANA MOISES PINTO,2007,G,Secondary Creator/Distributor,9.024282,-14.655399


In [133]:
careerBPM = dfBPM.groupby('Player')['BPM'].mean()
careerBPM = careerBPM.sort_values(ascending = False)
careerBPM.head(15)

Player
AMISHA CARTER            18.956086
MONIQUE COKER            10.625138
SHYLA HEAL                9.251625
TAMARA STOCKS             7.779064
BRITTANY WILKINS          7.731667
CHRISTELLE N'GARSANET     7.349094
SONJA MALLORY             6.780742
TERA BJORKLUND            6.449669
CELESTE TAYLOR            5.350904
TASHA BUTTS               4.679924
LEAH RUSH                 4.649267
KYSRE GONDREZICK          4.484401
DAVALYN CUNNINGHAM        4.147896
VICTORIA DUNLAP           3.599092
JESSICA BRUNGO            3.076168
Name: BPM, dtype: float64

In [135]:
byPosition2 = dfBPM.groupby('Pos')['BPM'].mean()
byPosition2

Pos
C        -5.226732
F        -4.936959
G        -4.858820
PowerF   -4.187847
SG       -4.639900
Name: BPM, dtype: float64

In [137]:
byArchetype2 = dfBPM.groupby('Archetype')['BPM'].mean()
byArchetype2

Archetype
Interior Primary Option         -3.857788
Perimeter-Oriented Big/Wing     -3.257006
Primary On-Ball Creator         -4.086462
Roller/Cutter                   -5.591724
Secondary Creator/Distributor   -5.152899
Spot-up Shooter                 -5.519760
Name: BPM, dtype: float64

In [139]:
dfBPM[dfBPM['Player'] == "MAYA MOORE"]

,Player,Year,Pos,Archetype,BPM,RAPM
63,MAYA MOORE,2014,F,Primary On-Ball Creator,1.335289,5.800633
165,MAYA MOORE,2015,F,Primary On-Ball Creator,-1.144136,3.782116
457,MAYA MOORE,2016,F,Primary On-Ball Creator,-1.148360,5.847532
460,MAYA MOORE,2012,F,Primary On-Ball Creator,-2.572627,5.492313
262,MAYA MOORE,2013,F,Primary On-Ball Creator,-2.796611,4.386548
278,MAYA MOORE,2017,F,Primary On-Ball Creator,-3.207312,1.619960
666,MAYA MOORE,2011,F,Primary On-Ball Creator,-3.332097,4.061656
217,MAYA MOORE,2018,F,Primary On-Ball Creator,-4.631300,4.156528


In [143]:
OBPM_coefficients.to_csv('OBPM_coefficients.csv', index=False)
DBPM_coefficients.to_csv('DBPM_coefficients.csv', index=False)
dfOBPM.to_csv('OBPM_results.csv', index=False)
dfDBPM.to_csv('DBPM_results.csv', index=False)
dfBPM.to_csv('BPM_results.csv', index=False)